<a href="https://colab.research.google.com/github/yangli2/rl_to_learn/blob/master/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from google.colab import drive
import tensorflow_datasets as tfds

GOOGLE_DRIVE = '/content/gdrive'
drive.mount(GOOGLE_DRIVE)
DRIVE_DIR = '{}/My Drive'.format(GOOGLE_DRIVE)
MNIST_DIR = '{}/mnist_dataset'.format(DRIVE_DIR)

mnist = tfds.image.mnist.MNIST(data_dir=MNIST_DIR)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
mnist = tfds.image.mnist.MNIST()
mnist.download_and_prepare(download_dir=MNIST_DIR)
dataset = mnist.as_dataset()